In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf

# Adjusted threshold for anomaly detection
ANOMALY_THRESHOLD = 1.3950 # A much smaller threshold is expected for single-frame MSE

# Ensure output folders exist
if not os.path.exists("output_frames"): # Re-added: Ensure output_frames directory exists
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"): # Folder for reconstructed image outputs (JPEGs)
    os.makedirs("reconstructed_frames")

frame_count_processed = 0
anomaly_count = 0

def mean_squared_loss(x1, x2):
    """
    Calculates the mean squared loss between two arrays.
    Returns the mean distance.
    This function is now used for single frames.
    """
    difference = x1.flatten() - x2.flatten()
    sq_difference = difference ** 2
    distance = np.sqrt(sq_difference.sum())
    n_samples = np.prod(x1.shape[1:])
    mean_distance = distance / n_samples
    return mean_distance

def combined_loss(y_true, y_pred):
    """
    Custom combined loss function for the Keras model.
    Assumes mean squared error is the primary component.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocess frame for the new model architecture.
    The new model expects better normalized inputs.
    """
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    gray = gray.astype(np.float32) / 255.0
    gray = (gray - 0.5) * 2
    return gray

# Load the trained Keras model (updated path for frame-by-frame model)
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model("../models/model.h5", custom_objects=custom_objects)
    print("Frame-by-frame model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model.h5' is in the '../models/' directory.")
    exit()

# Open the video file for processing
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or issue reading frame.")
        break

    frame_count_processed += 1

    original_frame_color = imutils.resize(frame, width=700, height=600)
    processed_frame = preprocess_frame_for_new_model(frame)
    model_input_frame = np.expand_dims(np.expand_dims(processed_frame, axis=0), axis=-1)
    output = model.predict(model_input_frame)

    # --- Save Model Prediction (Individual Reconstructed Frame as Image) ---
    reconstructed_frame_2d = output[0, :, :, 0]

    reconstructed_frame_display = cv2.normalize(
        reconstructed_frame_2d, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U
    )
    reconstructed_frame_display = cv2.resize(
        reconstructed_frame_display, (original_frame_color.shape[1], original_frame_color.shape[0])
    )
    cv2.imwrite(f"reconstructed_frames/frame_{frame_count_processed:05d}_reconstructed.jpg", reconstructed_frame_display)
    # --- End Save Model Prediction (Image) ---

    loss = mean_squared_loss(model_input_frame, output)
    loss_scaled = loss * 1000
    loss_display = f"{loss_scaled:.5f}"
    print(f"Frame {frame_count_processed} Loss: {loss_display}")

    display_frame = original_frame_color.copy()

    status_text = f"Normal: {loss_display}"
    color = (0, 255, 0) # Green for normal

    if loss_scaled > ANOMALY_THRESHOLD:
        status_text = f"Abnormal: {loss_display}"
        color = (0, 0, 255) # Red for abnormal
        anomaly_count += 1 # Increment anomaly count only when abnormal

    cv2.putText(display_frame, status_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    # Re-added: Save the annotated frame to output_frames
    if status_text.startswith("Abnormal"):
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}_abnormal.jpg", display_frame)
    else:
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}_normal.jpg", display_frame)


    cv2.imshow("Anomaly Detection", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if frame_count_processed % 100 == 0:
        print(f"Processed {frame_count_processed} frames, found {anomaly_count} anomalies so far.")

cap.release()
cv2.destroyAllWindows()

print(f"\nProcessing complete!")
print(f"Total frames processed: {frame_count_processed}")
print(f"Total anomalies detected: {anomaly_count}")
if frame_count_processed > 0:
    anomaly_rate = (anomaly_count / frame_count_processed * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No frames were processed, cannot calculate anomaly rate.")


Frame-by-frame model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 0s 107ms/step
Frame 1 Loss: 1.38392
1/1 [==============================] - 0s 17ms/step
Frame 2 Loss: 1.38264
1/1 [==============================] - 0s 17ms/step
Frame 3 Loss: 1.38165
1/1 [==============================] - 0s 19ms/step
Frame 4 Loss: 1.38130
1/1 [==============================] - 0s 17ms/step
Frame 5 Loss: 1.38069
1/1 [==============================] - 0s 16ms/step
Frame 6 Loss: 1.38025
1/1 [==============================] - 0s 16ms/step
Frame 7 Loss: 1.38002
1/1 [==============================] - 0s 17ms/step
Frame 8 Loss: 1.37835
1/1 [==============================] - 0s 17ms/step
Frame 9 Loss: 1.37753
1/1 [==============================] - 0s 17ms/step
Frame 10 Loss: 1.37680
1/1 [==============================] - 0s 17ms/step
Frame 11 Loss: 1.37565
1/1 [==============================] - 0s 16ms/step
Frame 12 Loss: 1.37188
1/1 [==============================] - 